In [ ]:
# @author Neill Kaipo Shikada
## ATLAS Institute - Design Methods Final Project
## Artist Statement
    ## Create a cooperative drawing program meant to use pose detection to allow drawing between an
    ## participant's hands. The distance between their hands dictates the radius while the mean between
    ## the hands is the centerpoint of the circle.
    
    ## Code inspired by Nicholas Renotte

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import uuid
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_mean(a,b):
    a = np.array(a) #First point
    b = np.array(b) #Second point
    
    # Calculate mean of the points
    x = (a[0]+b[0])/2
    y = (a[1]+b[1])/2
    
    return x,y

In [ ]:
def calculate_distance(a,b):
    a = np.array(a) #First point
    b = np.array(b) #Second point
    x1 = a[0]
    x2 = b[0]
    y1 = a[1]
    y2 = b[1]
    
    # Calculate the distance between the points
    d = ((((x2 - x1 )**2) + ((y2-y1)**2) )**0.5)
    
    return d

In [ ]:
def calculate_depth(z):
    
    depth = (z*-100) + 100
    
    return depth

In [ ]:
def extract_center(center_coordinates):    
    
    centerlist.append(center_coordinates)
       
    return centerlist

In [ ]:
def extract_radius(radius):    
    
    radiuslist.append(radius)
       
    return radiuslist

In [ ]:
def extract_color(depth):
    
    color = ((center_coordinates[0]/2), (depth-depth*0.5), (depth+depth*0.25))
    
    colorlist.append(color)
    
    return colorlist

In [ ]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video device")
# Adjust the window dimensions
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)

# Make Window fullscreen
window_name = "Cooperative Forms"
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

centerlist = []
radiuslist = []
colorlist = []

## Setup the mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Flip image horizontally
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        
        # Activates the pose detection function from mediapipe and stores those in an array in "results"
        results = pose.process(image)
        
        # Recolors image back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Extract Landmarks
            ri = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            li = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            rz = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].z
            lz = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].z
            rviz = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].visibility
            lviz = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].visibility
            
            # Calculate Mean
            mean = calculate_mean(ri, li)
            distance = calculate_distance(ri, li)
            depth = calculate_depth(rz)
            
            # Draw circle at the mean
            center_coordinates = tuple(np.multiply(mean, [960, 480]).astype(int))
            radius = int((distance*15)**2)
            #color = tuple(depth, (depth-(depth*1.25), 200)
            cv2.circle(image, center_coordinates, radius, (255,255,255), 1)
            
        except:
            pass
        
        
        try:
            if rviz >= 0.5 and lviz >= 0.5:
                extract_center(center_coordinates)
                extract_color(depth)
                if radius <= 100:
                    extract_radius(radius)
                    
            i = 0
            while i<len(centerlist):
                cv2.circle(image, centerlist[i], radiuslist[i], colorlist[i], -1)
                i += 1
        except:
            pass
        
        if cv2.waitKey(10) & 0xFF == ord(' '):
            centerlist = []
            radiuslist = []
            colorlist = []
        
        # Render the detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(150,150,150), thickness=1, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(150,150,150), thickness=1, circle_radius=2)
                                )
        
        cv2.imshow('Cooperative Forms', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'): 
            break
            
# Save our image  
cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
cv2.imshow('Hand Tracking', image)   

cap.release()
cv2.destroyAllWindows()